## Constraint Generator Demo

In [1]:
from llm_string.constraint_generator.evaluations.main_py import generate_functions
from llm_string.constraint_generator.evaluations.main_smt import generate_formal_constraints

In [2]:
from llm_string.constraints import ConstraintStore

constraint_store = ConstraintStore(file_path="../../../constraint_files/constraints_independent.csv")

### Generating Python Function Checkers

In [3]:
from omegaconf import OmegaConf

from llm_string.constraints import ConstraintStore

example_config = OmegaConf.create({
    "generator_type": "python",
    "generator_mode": "independent",
    "num_processes": 12,
    "max_retries_per_attempt": 2,
    "output_folder": "llm_string/constraint_generator/evaluations/results/",
})

constraint_store = ConstraintStore(file_path="../../../constraint_files/constraints_independent.csv")


results = generate_functions(0, example_config, constraint_store, ("gpt-4o-mini", "Name", True))

2025-06-14 13:10:57.891 | INFO     | llm_string.constraint_generator.core.python.batch_constraint_generator_agent:get_evaluator:35 - constraint=['The name shall only contain letters a-z, letters A-Z and space characters.'], attempt 1 invoking model.
2025-06-14 13:11:01.998 | INFO     | llm_string.constraint_generator.core.python.batch_constraint_generator_agent:get_evaluator:35 - constraint=['The name shall contain at least one space character.'], attempt 1 invoking model.
2025-06-14 13:11:05.904 | INFO     | llm_string.constraint_generator.core.python.batch_constraint_generator_agent:get_evaluator:35 - constraint=['The name shall not end with a space character.'], attempt 1 invoking model.
2025-06-14 13:11:11.130 | INFO     | llm_string.constraint_generator.core.python.batch_constraint_generator_agent:get_evaluator:35 - constraint=['The name shall not start with a space character.'], attempt 1 invoking model.
2025-06-14 13:11:16.089 | INFO     | llm_string.constraint_generator.core.py

In [4]:
for result in results:
    print("NL Constraint:", result[0])
    print("Python Function:", '\n', result[1].encode('utf-8').decode('utf-8'))
    print('\n')

NL Constraint: The name shall only contain letters a-z, letters A-Z and space characters.
Python Function: 
 def this_function(s: str) -> bool:
    ''' Check if the name shall only contain letters a-z, letters A-Z and space characters. '''
    return all(char.isalpha() or char.isspace() for char in s)


NL Constraint: The name shall contain at least one space character.
Python Function: 
 def this_function(s: str) -> bool: 
    ''' Check if the name contains at least one space character.'''
    return ' ' in s


NL Constraint: The name shall not end with a space character.
Python Function: 
 def this_function(s: str) -> bool: 
    ''' Check if the name does not end with a space character.'''
    return not s.endswith(' ')


NL Constraint: The name shall not start with a space character.
Python Function: 
 def this_function(s: str) -> bool: 
    ''' Check if the name does not start with a space character.'''
    return not s.startswith(' ')


NL Constraint: The first character in the na

### Generating SMT Constraint Checkers

In [5]:
from omegaconf import OmegaConf

example_config = OmegaConf.create({
    "generator_type": "smt",
    "generator_mode": "independent",
    "num_processes": 12,
    "max_retries_per_attempt": 2,
    "output_folder": "llm_string/constraint_generator/evaluations/results/",
})


results = generate_formal_constraints(0, example_config, constraint_store, ("gpt-4o-mini", "Name", True))

2025-06-14 13:11:26.116 | INFO     | llm_string.constraint_generator.core.batch_constraint_generator_agent:_execute_evaluator_step:116 - Attempt 1: sending constraint to the LLM: ['The name shall only contain letters a-z, letters A-Z and space characters.']
2025-06-14 13:11:29.484 | INFO     | llm_string.constraint_generator.core.batch_constraint_generator_agent:_execute_evaluator_step:131 - Received constraints from the LLM: variables=['name'] constraint=['(str.in.re name (re.* (re.union (re.range "a" "z") (re.range "A" "Z") (str.to.re " "))))']
2025-06-14 13:11:29.489 | INFO     | llm_string.constraint_generator.core.helpers.solver_helpers:_update_solver_with_smt_lib2_constraints:102 - Checking if the constraints are satisfiable.
2025-06-14 13:11:29.491 | INFO     | llm_string.constraint_generator.core.batch_constraint_generator_agent:_execute_evaluator_step:137 - Successfully created evaluator. Returning evaluator.
2025-06-14 13:11:29.790 | INFO     | llm_string.constraint_generator

In [6]:
for result in results:
    print("NL Constraint:", result[0])
    print("SMT Constraint:", result[1])
    print('\n')

NL Constraint: The name shall only contain letters a-z, letters A-Z and space characters.
SMT Constraint: (str.in.re s (re.* (re.union (re.range "a" "z") (re.range "A" "Z") (str.to.re " "))))


NL Constraint: The name shall contain at least one space character.
SMT Constraint: (str.contains s " ")


NL Constraint: The name shall not end with a space character.
SMT Constraint: (not (= (str.at s (- (str.len s) 1)) " "))


NL Constraint: The name shall not start with a space character.
SMT Constraint: (not (= (str.at s 0) " "))


NL Constraint: The first character in the name shall be capitalized.
SMT Constraint: (and (>= (str.to_int (str.at s 0)) (str.to_int "A")) (<= (str.to_int (str.at s 0)) (str.to_int "Z")))


NL Constraint: Any character in the name following a space character shall be capitalized.
SMT Constraint: (forall ((i Int)) (=> (and (>= i 0) (< i (str.len s))) (=> (= (str.at s i) " ") (=> (< (+ i 1) (str.len s)) (isUpper (str.at s (+ i 1)))))))




### Generation Results from our Experiments

In [ ]:
%%cmd
cd ../../../
pip install -r requirements.txt
python llm_string/constraint_generator/evaluations/show_results.py